# Thermalization

In [2]:
%matplotlib widget 

# global import
import numpy as np
import pandas as pd

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
# local import
import lppydsmc as ld

seed = 1111
np.random.seed(1111)

## General theory

Apparently there could be a analytical solution for the characteristic time of relaxation towards equilibrium.

The idea here is to initialize a 0D box and one species of a monoatomic atom (for example : Iodine). 

Then, the translational energy along axis $y$ and $z$ (for example) relaxes towards the equilibrium. To do that, the translasional energy along the $x$ axis is reset at each time step to a give equilibrium and DSMC collisions are performed.

Parameters :
 - temperature : $300$ K
 - density : $3.2 \times 10^{19}$ $m^{-3}$
 - species : Iodine [I]
 
The code does not allow, by default, to reinitialize at each time step a species or one of its component. In addition, advection and collisions with walls are not needed.
Thus the choice that was made was to use the basic functions and code it from there.

In [5]:
def temperature_evolution(data, mass):
    from lppydsmc.utils.physics import BOLTZMAN_CONSTANT
    # speed norm
    data['v2'] = data['vx']*data['vx']+data['vy']*data['vy']+data['vz']*data['vz']
    data['v'] = np.sqrt(data['v2'])

    # drift 
    v_mean = data.groupby(data.index).mean()
    v_mean['drift2'] = v_mean['vx']*v_mean['vx']+v_mean['vy']*v_mean['vy']+v_mean['vz']*v_mean['vz']
    v_mean['drift'] = np.sqrt(v_mean['drift2'])

    # 3/2 k T = 1/2 m (<v²>-|<v>|²)
    temperature = mass/(3.*BOLTZMAN_CONSTANT)*(v_mean['v2']-v_mean['drift2'])
    # temperature = mass/(3.*BOLTZMAN_CONSTANT)*((v_mean['v']-v_mean['drift'])**2)

    return temperature

def variance_speed_evolution(data):
    data['v2'] = data['vx']*data['vx']+data['vy']*data['vy']+data['vz']*data['vz']
    data['v'] = np.sqrt(data['v2'])
    data_var = data.groupby(data.index).var()
    return data_var['v']

def translational_energy(data, mass):
    data['tx'] = 0.5*mass*data['vx']*data['vx']
    data['ty'] = 0.5*mass*data['vy']*data['vy']
    data['tz'] = 0.5*mass*data['vz']*data['vz']
    
    groups = data[['tx','ty','tz']].groupby(data.index).mean()
    
    return groups

In [3]:
def thermalization(mass, radius, quantity, time_step, iterations, temperature = 300, seed = None):
    import pandas as pd
    import lppydsmc as ld
    import numpy as np
    from tqdm import tqdm
    from pathlib import Path
    np.random.seed(seed)
    
    df = pd.DataFrame(columns = ['vx','vy','vz']) # index will be the iteration
    
    points = 1e-3*np.array([[0,0],[0,1],[1,1],[1,0]])
    positions = ld.initialization.particles.uniform_position(quantity, points)
    
    velocities = np.zeros((quantity, 3))
    vel_std = ld.utils.physics.gaussian(temperature, mass)
    velocities[:,0] = np.random.normal(loc=0.0, scale=vel_std, size = quantity)
    
    arrays = [np.concatenate((positions, velocities), axis = 1)]
    
    grid = np.array([np.zeros((quantity,2))]) # 1 cell
    grid[0][:,1] = np.array([k for k in range(quantity)])
    currents = np.array([quantity])
    averages = np.array([quantity])
    
    cross_section = np.pi*4*radius*radius
    cross_sections = np.array([[cross_section]])

    max_proba = np.array([cross_section*ld.utils.physics.maxwellian_mean_speed(temperature, mass)])

    target_density = 3.2e19
    cell_volume = 1e-9
    particles_weight = target_density*cell_volume/quantity
    remains_per_cell = np.array([0])
    masses = np.array([mass])
    monitoring = None
    group_fn = None
    saver = ld.data.saver.Saver(Path(''), 'thermalization_monitoring_2.h5')
    
    period_saving = 10
    with saver.__enter__() as store :
        df = df.append(pd.DataFrame(arrays[0][:,2:], index=[0]*quantity, columns = ['vx','vy','vz']))
        for iteration in tqdm(range(1,iterations+1)) :
            results = ld.collision.handler_particles_collisions(arrays, grid, currents, time_step, \
                averages, max_proba, cross_sections, cell_volume, particles_weight, remains_per_cell, masses, monitoring = monitoring, group_fn = group_fn)

            df = df.append(pd.DataFrame(arrays[0][:,2:], index=[iteration]*quantity, columns = ['vx','vy','vz']))

            # resetting vx
            arrays[0][:,2] = np.random.normal(loc=0.0, scale=vel_std, size = quantity)
            
            if(iteration%period_saving==0):
                # print('Collisions : {}'.format(results))
                saver.save(it = iteration, append = {'df':df})
                df = pd.DataFrame(columns = ['vx','vy','vz'])

In [8]:
mass = 2.16e-25 # kg
time_step = 1e-5
radius = 2e-10
J_TO_EV = 1/1.6e-19
# thermalization(mass = mass, radius = radius, quantity = int(1e5), time_step = time_step, iterations = 1000, temperature = 300, seed = 1111)

In [9]:
store = pd.HDFStore('thermalization_monitoring_2.h5')
df = store['df']
df.head()

,vx,vy,vz
0,61.576818,0.0,0.0
0,-78.067974,0.0,0.0
0,65.671737,0.0,0.0
0,-43.097120,0.0,0.0
0,-32.607050,0.0,0.0


In [10]:
te = translational_energy(df, mass)

In [11]:
temperatures = temperature_evolution(df, mass) 

In [17]:
fig, ax = plt.subplots(constrained_layout = True)
ax.set_title(r'$T_f =$' + '{:.2f} K'.format(np.mean(temperatures.values[500:])))
ax.plot(te.index*time_step, te['tx']*J_TO_EV, color = 'r', label = 'Tx')
ax.plot(te.index*time_step, te['ty']*J_TO_EV, color = 'g', label = 'Ty')
ax.plot(te.index*time_step, te['tz']*J_TO_EV, color = 'b', label = 'Tz')
ax.legend(loc = 'right', fontsize = 14)
ax.set_xlabel('t (s)', fontsize = 14)
ax.set_ylabel('translational energy (eV)', fontsize = 14);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
df['t'] = df['tx']+df['ty']+df['tz']

In [14]:
tf = df['t'].loc[df.index > 800]

In [15]:
target_temperature = 300
target_density = 3.2e19
factor = ld.utils.physics.BOLTZMAN_CONSTANT*target_temperature
f = np.exp(-tf/factor)

In [18]:
pre_factor = 1  # target_density # target_density*(mass/(factor*np.pi*2))**(3/2)
fig, ax = plt.subplots(constrained_layout=True)
ax.set_yscale('log')
ax.scatter(tf*J_TO_EV, pre_factor*f, s = 0.1);
ax.set_xlabel('Translational energy (eV)', fontsize=14)
ax.set_ylabel(r'Probability density $f(E_t, T)$', fontsize=14);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …